<em style="text-align:center">Copyright Iván Pinar Domínguez</em>

## Importar librerías iniciales e instancia de modelo de chat

In [1]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate,ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.agents import load_tools,initialize_agent,AgentType,create_react_agent,AgentExecutor
f = open('/home/iabd/clave_gpt.txt')
api_key = f.read().strip()
llm = ChatOpenAI(openai_api_key=api_key,temperature=0) #Recomendable temperatura a 0 para que el LLM no sea muy creativo, vamos a tener muchas herramientas a nuestra disposición y queremos que sea más determinista

# CASO DE USO 1

## Creamos nuestra herramienta personalizada 

In [2]:
from langchain.agents import tool

In [3]:
@tool
def persona_amable (text: str) -> str:
    '''Retorna la persona más amable. Se espera que la entrada esté vacía "" 
    y retorna la persona más amable del universo'''
    return "Miguel Celebres"

## Definimos las herramientas a las que tendrá acceso el agente y ejecutamos

### Ejemplo sin función personalizada 

In [5]:
tools = load_tools(["wikipedia","llm-math",],llm=llm)

In [6]:
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

/tmp/ipykernel_7653/790863316.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)


In [18]:
agent.invoke("¿Quién es la persona más amable del universo?")



> Entering new AgentExecutor chain...
I should use the persona_amable function to find out who is the kindest person in the universe.
Action: persona_amable
Action Input: ""
Observation: Miguel Celebres
Thought:I now know the final answer
Final Answer: Miguel Celebres

> Finished chain.


{'input': '¿Quién es la persona más amable del universo?',
 'output': 'Miguel Celebres'}

### Ejemplo con función personalizada 

In [8]:
tools = tools + [persona_amable]

In [9]:
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

In [10]:
agent.invoke("¿Quién es la persona más amable del universo?")



> Entering new AgentExecutor chain...
Let's use the persona_amable function to find out.
Action: persona_amable
Action Input: ""
Observation: Miguel Celebres
Thought:I need to verify this information.
Action: wikipedia
Action Input: Miguel Celebres
Observation: Page: Mocidade Independente de Padre Miguel
Summary: The Grêmio Recreativo Escola de Samba Mocidade Independente de Padre Miguel is a samba school of the city of Rio de Janeiro, being located on Rua Coronel Tamarindo, in the neighborhood of Padre Miguel.



Page: George W. Bush
Summary: George Walker Bush (born July 6, 1946) is an American politician who served as the 43rd president of the United States from 2001 to 2009. A member of the Bush family and the Republican Party, he was the 46th governor of Texas from 1995 to 2000. 
The eldest son of the 41st president, George H. W. Bush, he flew warplanes in the Texas Air National Guard in his twenties. After graduating from Harvard Business School in 1975, he worked in the oil in

{'input': '¿Quién es la persona más amable del universo?',
 'output': 'Miguel Celebres'}

# CASO DE USO 2: API interna

In [11]:
@tool
def nombre_api_interna(text: str) -> str:
    '''Conecta a la API_xx que realiza la tarea xx, debes usar esta API Key'''
    ##Definir conexión a la API interna y devolver un resultado
    return resultado

# CASO DE USO 3: Consultar hora actual

In [ ]:
# Solicitud con las herramientas actuales no proporciona el resultado que queremos (da error)
agent.invoke("¿Cuál es la hora actual?")



> Entering new AgentExecutor chain...
I should use a calculator to determine the current time.
Action: Calculator
Action Input: current time

ValueError: LLMMathChain._evaluate("
datetime.now().time()
") raised error: Expression datetime.now().time() has forbidden control characters.. Please try again with a valid numerical expression

## Creamos nuestra función personalizada 

In [13]:
from datetime import datetime

In [14]:
@tool
def hora_actual(text: str)->str:
    '''Retorna la hora actual, debes usar esta función para cualquier consulta sobre la hora actual. Para fechas que no sean
    la hora actual, debes usar otra herramienta. La entrada está vacía y la salida retorna una string'''
    return str(datetime.now())

In [15]:
tools = tools + [hora_actual]

In [16]:
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

In [17]:
# Solicitud con las herramientas actuales SÍ proporciona el resultado que queremos
agent.invoke("¿Cuál es la hora actual?")



> Entering new AgentExecutor chain...
I should use the hora_actual function to get the current time.
Action: hora_actual
Action Input: ""
Observation: 2025-02-16 16:49:39.250045
Thought:I have the current time.
Final Answer: 2025-02-16 16:49:39.250045

> Finished chain.


{'input': '¿Cuál es la hora actual?', 'output': '2025-02-16 16:49:39.250045'}